In [1]:
import os
import json
#import shutil

from library_dicom.dicom_processor.model.Series import Series
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.series import get_series_object


In [2]:
series_paths = get_series_path("/media/deeplearning/SAMSUNG/AHL2011")
export_folder = '/media/deeplearning/AHL/JSON'

In [3]:
len(series_paths)

10088

In [4]:
#get every serie uid from second clean up 
liste_new_uid = []
for path in series_paths : 
    uid = path.split('/')[-1]
    #print(uid)
    if 'imagys' not in uid : 
        #print(uid)
        liste_new_uid.append(uid)

In [5]:
len(liste_new_uid)

9774

In [6]:
liste_new_uid[0]

'1.3.12.2.1107.5.1.4.48467.30000013010207240617100011289'

In [7]:
#get series uid from previous cleanup 

json_path = '/media/deeplearning/CLé USB/AHL_JSON'
liste_json = os.listdir(json_path)
liste_uid = []
for name in liste_json : 
    uid = name[:-5]
    liste_uid.append(uid)
    #print(uid)

In [8]:
len(liste_uid)

3976

In [9]:
liste_uid[0]

'1.3.46.670589.28.2.10.4.9167.27815.2.2040.0.1344324331'

In [10]:
#compare two dataset : 
remove = []
for uid in liste_new_uid : 
    for old_uid in liste_uid : 
        if old_uid == uid : 
            remove.append(uid)


In [11]:
len(remove)

3893

In [12]:
treated = []
for path in series_paths : 
    for uid in remove : 
        if uid in path : 
            treated.append(path)
            


In [13]:
print(treated[0])

/media/deeplearning/SAMSUNG/AHL2011/AHLGroupe13/Batch1/13011101091017/PET0/1.3.12.2.1107.5.1.4.48467.30000013010207240617100011289


In [14]:
for serie in treated : 
    try : 
        series_paths.remove(serie)
    except Exception as err : 
        print(err)

list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remove(x): x not in list
list.remov

In [15]:
len(series_paths)

6192

In [18]:
len(liste_uid)

3976

In [20]:
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/', 'serie_uid_treated', liste_uid)

In [5]:
index_1 = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/Relevance/index_RELEVANCE.json'
index_2 = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/Relevance/index_part_2.json'
index_3 = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/Relevance/index_RELEVANCE_part3.json'

dataset_1 = []
with open(index_1) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_1.append(info)

dataset_2 = []
with open(index_2) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_2.append(info)

dataset_3 = []
with open(index_3) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_3.append(info)


liste_index = dataset_1 + dataset_2 + dataset_3

treated = []
for serie_path in series_paths : 
    serie_uid = serie_path.split('/')[-1]
    if serie_uid in liste_index : 
        treated.append(serie_path)
print(len(treated))

0


In [10]:
for serie in treated : 
    series_paths.remove(serie)

In [23]:
export_folder = '/home/deeplearning/AHL/JSON'

not_image_folder=[]
non_valid_series=[]
non_attenuation_corrected=[]
empty_folders = []
secondary_series = []
localizer_series = []

for serie_path in series_paths:
    print(series_paths.index(serie_path))
    if(len(os.listdir(serie_path)) == 0):
        empty_folders.append(serie_path)
        continue
    try:
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        remove_bi_file(serie_path)
        if( dicom_serie.is_image_modality() == False):
            not_image_folder.append(serie_path)
        elif (dicom_serie.is_series_valid() == False):
            non_valid_series.append(serie_path)
        elif(type(dicom_serie) == SeriesPT and dicom_serie.is_corrected_attenuation() == False):
            non_attenuation_corrected.append(serie_path)
        elif(dicom_serie.is_localizer_series() == True):
            localizer_series.append(serie_path)
        else:

            write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], dicomsInfo)
            #print("json_file")
    except Exception as err:
        print(err)
        print(dicomsInfo)

xelSpacing': ["4.06271", "4.06271"], 'ImagePosition': ["-335.94868982376", "-523.73184001984", "7.0"], 'ImageOrientation': ["1.0", "0.0", "0.0", "0.0", "1.0", "0.0"], 'RescaleSlope': "0.527461", 'RescaleIntercept': "0.0", 'SOPInstanceUID': '1.3.12.2.1107.5.1.4.44873.30000013041205203910900003837'}, 'radiopharmaceutical': {'RadionuclideHalfLife': "6586.2", 'TotalDose': "431790008.54492", 'RadiopharmaceuticalStartDateTime': 'Undefined', 'RadiopharmaceuticalStartTime': '113000'}, 'pet_correction': ['NORM', 'DTIM', 'RADL', 'DECY'], 'philips_tags': {'PhilipsSUVFactor': 'Undefined', 'PhilipsBqMlFactor': 'Undefined'}}
6110
6111
6112
6113
6114
(0028, 0030)
{'series': {'Manufacturer': 'SIEMENS', 'Modality': 'CT', 'SeriesDate': '20130408', 'SeriesTime': '130555', 'SeriesDescription': 'CT M INF 3.0 B31s', 'SeriesInstanceUID': '1.3.12.2.1107.5.1.4.44873.30000013040806565018700002765', 'SeriesNumber': "3", 'AcquisitionDate': '20130408', 'AcquisitionTime': '130552.314', 'NumberOfSlices': 'Undefined'

In [24]:
#print(not_image_folder)
#print(non_valid_series)
#print(non_attenuation_corrected)
#print(empty_folders)
#print(secondary_series)
#print(localizer_series)

print(len(not_image_folder))
print(len(non_valid_series))
print(len(non_attenuation_corrected))
print(len(empty_folders))
print(len(secondary_series))
print(len(localizer_series))

0
65
1752
188
0
323


for localizer_serie in localizer_series:
    shutil.rmtree(localizer_serie)

for not_image in not_image_folder:
    shutil.rmtree(not_image)

for non_attenuation in non_attenuation_corrected:
    shutil.rmtree(non_attenuation)

for non_valid_serie in non_valid_series : 
    shutil.rmtree(non_valid_serie)

for secondary_serie in secondary_series : 
    shutil.rmtree(secondary_serie)

for empty_folder in empty_folders:
    shutil.rmtree(empty_folder)

In [25]:
write_json_file("/home/deeplearning/AHL", "wrong Series AHL", json.dumps(non_valid_series))